In [6]:
import lark
from lark import Lark, Transformer, v_args

In [827]:
kumir_robot = """
    start: "использовать" "Робот"i alg+

    alg: "алг" name body 

    name: CHAR | CHAR name
    
    body: "нач" command+ "кон"

    command: cycle | (action";")* action | if_expression 

    action: /вверх/i | /вниз/i | /влево/i | /вправо/i | /закрасить/i

    cycle: "нц" (NUM "раз" | while_cycle) command+ ("кц" | "кц_при" condition)

    while_cycle: "пока" condition

    condition: NOT? direction wall

    wall: /свободно/i | /стена/i

    direction: /слева/i | /справа/i | /сверху/i | /снизу/i

    if_expression: "если" condition "то" command+ (else_expression | "все")

    else_expression: "иначе" command+ "все"

    NOT: "не"

    CHAR: /[|а-яА-Я0-9" "]+/

    VAR: /[a-zA-Z]/

    NUM: /[0-9]+/

%import common.NEWLINE
%import common.WS_INLINE
%ignore WS_INLINE
%ignore NEWLINE
"""

In [828]:
class RobotInterpreter(Transformer):

    def start(self, items):
        return ''

    def alg(self, items):
        name = items[0]
        body = items[1]
        print(f"Парсинг алгоритма{name} завершен успешно.")
        return body

    def name(self, items):
        return ''.join(str(item) for item in items)

    def action(self, token):
        act = str(token[0]).lower()
        print(f"Обнаружено действие: {act}")

    def cycle(self, items):
        cycle_type = items[0]
        commands = items[1]
        if cycle_type[0] == 'while':
            print(f"Обнаружен цикл типа пока")
            return ('while', cycle_type[1], commands)
        else:
            count = cycle_type
            print(f"Обнаружен цикл типа повторение {count} раз")
            return ('repeat', count, commands)

    def while_cycle(self, items):
        condition = items[0]
        return ('while', condition)

    def condition(self, items):
        if len(items)==2:
            negation = False
            direction = items[0]
            wall = items[1]
        else:
            negation = True
            direction = items[1]
            wall = items[2]
        if negation:
            print(f"Обнаружено условие: {direction} не {wall}")
        else:
            print(f"Обнаружено условие: {direction} {wall}")

    def wall(self, token):
        return str(token[0]).lower()

    def direction(self, token):
        return str(token[0]).lower()

    def NOT(self, token):
        return str(token)

пробую вложенность

In [829]:
from lark import Transformer

class RobotInterpreter(Transformer):

    def start(self, items):
        return {"algorithms": items}

    def alg(self, items):
        name = items[0]
        body = items[1]
        return {"type": "algorithm", "name": name, "body": body}

    def name(self, items):
        return ''.join(items)

    def body(self, items):
        return {"commands": items}

    def command(self, items):
        return items

    def action(self, token):
        return {"type": "action", "name": str(token[0]).lower()}

    def cycle(self, items):
        if isinstance(items[0], tuple) and items[0][0] == 'while':
            _, condition = items[0]
            commands = items[1:]
            return {
                "type": "while",
                "condition": condition,
                "body": commands
            }
        else:
            count = items[0]
            commands = items[1:]
            return {
                "type": "repeat",
                "count": int(count),
                "body": commands
            }

    def while_cycle(self, items):
        return ('while', items[0])

    def condition(self, items):
        if len(items) == 2:
            negation = False
            direction, wall = items
        else:
            negation = True
            _, direction, wall = items
        return {
            "negation": negation,
            "direction": direction,
            "wall": wall
        }

    def wall(self, token):
        return str(token[0]).lower()

    def direction(self, token):
        return str(token[0]).lower()

    def NOT(self, token):
        return str(token[0])

    def if_expression(self, items):
        condition = items[0]
        commands = []
        else_branch = []

        found_else = False
        for item in items[1:]:
            if isinstance(item, dict) and item.get("type") == "else":
                else_branch = item["body"]
                found_else = True
            else:
                commands.append(item)

        return {
            "type": "if",
            "condition": condition,
            "then": commands,
            "else": else_branch if found_else else None
        }

    def else_expression(self, items):
        return {"type": "else", "body": items}


In [830]:
parser = Lark(kumir_robot, start='start', parser='lalr')

In [831]:
input_string = """
использовать Робот 
алг ура
нач
нц пока сверху свободно 
  если не справа свободно то
    закрасить
  все
кц
закрасить
нц 3 раз
влево
кц
кон
"""

In [832]:
try:
    parsed_tree = parser.parse(input_string)
    print("Parsed Tree:", parsed_tree.pretty())
except Exception as e:
    print(f"Ошибка при разборе ввода: {e}")

Parsed Tree: start
  alg
    name	 ура
    body
      command
        cycle
          while_cycle
            condition
              direction	сверху
              wall	свободно
          command
            if_expression
              condition
                не
                direction	справа
                wall	свободно
              command
                action	закрасить
      command
        action	закрасить
      command
        cycle
          3
          command
            action	влево



In [ ]:
from pprint import pprint
try:
    transformed_result = RobotInterpreter().transform(parsed_tree)
    pprint(transformed_result)
except Exception as e:
    print(f"Ошибка при разборе ввода: {e}")

{'algorithms': [{'body': {'commands': [[{'body': [[{'condition': {'direction': 'справа',
                                                                  'negation': True,
                                                                  'wall': 'свободно'},
                                                    'else': None,
                                                    'then': [[{'name': 'закрасить',
                                                               'type': 'action'}]],
                                                    'type': 'if'}]],
                                         'condition': {'direction': 'сверху',
                                                       'negation': False,
                                                       'wall': 'свободно'},
                                         'type': 'while'}],
                                       [{'name': 'закрасить',
                                         'type': 'action'}],
                          

In [775]:
input_string = """
использовать Робот
алг из А в Б
нач
    вправо
    нц 5 раз
        вверх; вверх; вправо; вниз; вниз; вправо
    кц
кон
"""

In [369]:
try:
    parsed_tree = parser.parse(input_string)
    print("Parsed Tree:", parsed_tree.pretty())
except Exception as e:
    print(f"Ошибка при разборе ввода: {e}")


Parsed Tree: start
  alg
    name	 из А в Б
    body
      command
        action	вправо
      command
        cycle
          5
          раз
          command
            action	вверх
            action	вверх
            action	вправо
            action	вниз
            action	вниз
            action	вправо



In [370]:
try:
    transformed_result = RobotInterpreter().transform(parsed_tree)
    print(transformed_result)
except Exception as e:
    print(f"Ошибка при разборе ввода: {e}")

Обнаружено действие: вправо
Обнаружено действие: вверх
Обнаружено действие: вверх
Обнаружено действие: вправо
Обнаружено действие: вниз
Обнаружено действие: вниз
Обнаружено действие: вправо
Обнаружен цикл типа повторение 5 раз
Парсинг алгоритма из А в Б завершен успешно.



In [375]:
input_string = """
использовать Робот
алг ААА
нач
    вправо
    нц 5 раз
        если сверху свободно то
            вверх; вверх; вправо; вниз; вниз; вправо
        иначе
            закрасить
            вправо
        все
    кц
кон
"""

In [376]:
try:
    parsed_tree = parser.parse(input_string)
    print("Parsed Tree:", parsed_tree.pretty())
except Exception as e:
    print(f"Ошибка при разборе ввода: {e}")


Parsed Tree: start
  alg
    name	 ААА
    body
      command
        action	вправо
      command
        cycle
          5
          раз
          command
            if_expression
              condition
                direction	сверху
                wall	свободно
              command
                action	вверх
                action	вверх
                action	вправо
                action	вниз
                action	вниз
                action	вправо
              else_expression
                command
                  action	закрасить
                command
                  action	вправо



In [377]:
try:
    transformed_result = RobotInterpreter().transform(parsed_tree)
    print(transformed_result)
except Exception as e:
    print(f"Ошибка при разборе ввода: {e}")

Обнаружено действие: вправо
Обнаружено условие: сверху свободно
Обнаружено действие: вверх
Обнаружено действие: вверх
Обнаружено действие: вправо
Обнаружено действие: вниз
Обнаружено действие: вниз
Обнаружено действие: вправо
Обнаружено действие: закрасить
Обнаружено действие: вправо
Обнаружен цикл типа повторение 5 раз
Парсинг алгоритма ААА завершен успешно.

